# Statistical birds eye view of the contents in an AiiDAdb

This is the first of two deliverable for the SiSc-Lab2020 project.

Authors = 

Supervisors: Dr. Jens Bröder, Dr. Daniel Wortmann, Johannes Wasmer, Prof. Dr. Stefan Blügel.

# Instructions by supervisors

## Jens

You have to implement this notebook.

In the end only text (markdown) cells and output results of code cells should be seen if one hides the code cells (hide_code extension).

That can easily exported into a nice pdf file (google it, probably will find sth with `nbconvert`).

Also the amount of python code in this notebook should be minimal.

Rather, export the functions you use to python file(s) and import them here (hide complexity).

Optional dump query results in a file, from which results will be reread for speed, i.e cache results.

## Johannes



After talking with Jens about it, here are some additional thoughts on the code structure and implementation, for both deliverables.

The **primary** goal is of course that the code should work, produce nice output (and helpful error messages), obviously.

The **secondary** goal is speed. How long do you expect your code to run on a dataset of a given size? Are there multiple paths to a goal, but with differing performance?

You can break the runtime down into several steps: data acquisition, data transformation (or preprocessing), data analysis, data visualization. In this project, we will rename/replace these steps to: **querying, de-/serialization, analysis=visualization**.

**Querying the database.** Performance considerations:
- Performance measurement: use the magics `%time` and `%timeit`.
- Query evaluations: queries (in general) use 'lazy evaluation'.
  - *Query building* methods build the query but do not execute it. These are chainable methods like `append()`, `get_outgoing()`, etc.
  - *Query execution* methods send the query to the database to be evaluated. There are two kinds:
    - non-iterator methods: e.g. `all()`, `first()`, etc. These return a result `list`: all items are loaded into memory.
    - iterator methods: e.g. `iterall()`, `iterdict()`. These return a result `Generator`: only one item at a time is loaded into memory.
    
**De-/serialization**, i.e. writing and reading it to/from a file. *Keep in mind: if you come to the conclusion this is unnecessary, then justify it!* Considerations:
- Necessity: we assume 'yes'. So you need serialization/deserialization routine(s).
- Code design: we recommend to write a serializer that moves *all* data needed from aiida to file (perform query & serialization). Then the visualization methods are decoupled from aiida and load data from that file. Advantages: a) only needs to be called when data in database changed, b) similar queries for different visualizations can be performed only once. One design option is this:
  ```python
  serialize = sisclab.Serializer(profile)
  serialize.to_file(filepath)
  visualize = sisclab.Visualizer(filepath)
  visualize.histogram(cumulative=True, plot_options)
  # plots histogram
  ```
- Serialization format: there are two practical options (maybe more):
  - `dict`: tree-like. JSON format. One `dict` per file. choose key-value (nested?) based on use-case. in general, `uuid` is a good key.
  - `pandas.Dataframe`: could be preferrable in some cases.
- Serialization location:
  - one file or several files?
  - we recommend to de/serialize from/to `sisclab/data/` folder. It is included in the project's `.gitignore` file, so nothing in it gets committed to/from git (git is for code, not for data; the code generates the data).
- Transformation:
  - if needed, decide where to put needed data transformations (before serialization or after deserialization) to minimize them.
- Deserialization: 
  - a class (as above) might help to define the deserialization format only once for all visualization methods.
  

**Visualization**:
- Prefer `bokeh` to `matplotlib` or other libs wherever possible, unless you have a good justification.
- In `D1`, static plots are okay, interactive plots are a bonus.
- Lists results (when plot is overkill) will look nicer in a notebook if they are a `pandas.Series` or `pandas.Dataframes`.
- Think about function signatures. Can you generalize them to make a nice interface? For example, a signature for SubtaskD1.c might look like this:
  ```python
  def node_type_summary(user_list : list = [], node_basetype : Node = Data,
                        chart_type : bokeh.chart_type = bokeh.pie_chart, plot : bool = True):
    """
    :param user_list: list of users. empty list = all users = default.
    :param node_basetype: subdivides chart into subtypes. Valid base type examples: ProcessNode, CalculationNode, WorkflowNode, Data, ArrayData.
    :param chart_type: bokeh visualization type. pie chart = default.
    :param plot: True: show plot, don't return data. False: don't plot, return data.
    :return: stats: a dictionary {node_subtype : node_count}, insertion-order sorted in descending order.
    :rtype: dict.
    """
  ```


## Remarks Johannes

Date: 07.01.2020.

Here are comments that pertain to multiple or all subtask solutions in their current state.

- Compliment: Nice how you reuse a single projection query for `D1.a-d`!
- Request: 
- Request: Next steps:
  - put code into functions now such that each sub-task is a one-liner function call, and all read from serialized data.
  - produce clean code:
    - please adhere to a good Python Style Guide, like e.g. the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html) or [PEP 8](https://www.python.org/dev/peps/pep-0008/). There are tools that automate this to some extent. But manually needed improvement is especially:
      - [Section 3.16 Naming](https://google.github.io/styleguide/pyguide.html#s3.16-naming) for variable naming.
      - [Section 3.8 Comments and docstrings](https://google.github.io/styleguide/pyguide.html#s3.8-comments-and-docstrings). But use the [sphinx/ResT docstring style](https://queirozf.com/entries/python-docstrings-reference-examples), not the one from Google.
  - provide timing functionality. maybe e.g. like a copy of the end-user notebook with timing instrumentation around the calls and summary/plotting at the end or something. So it can be run on datasets of different sizes and then compared for report.
- Question: Who did which subtask?
  - `D1.a`: 
  - `D1.b`: 
  - `D1.c`: 
  - `D1.d`: 
  - `D1.e`: 
  - `D1.f`: 
  - `D1.g`: 
  - `D1.h`: 
  - `D1.i`: 
  - `D2.a`: 
  - `D2.b`: 
- Suggestion / Request (@Jens?): Plots:
  - add export functionality for single plots --> pdf/png/...?

- Just thinking aloud:
  - Packaging: 
    - which lib verisons do you use? i use py 3.7, bokeh 1.4.0 here. thinking about compatibility here (pypi `setup.py`, pip `requirements.txt`, conda `environment.yml`). github has some automated packaging support. have a look [here](https://github.com/JuDFTteam/aiida-jutools/actions).
  - Deployment:
    - maybe start thinking about notebook deployment for end users, try out tools: notebook-based: appmode, voila, osscar hide code cells, ...; dashboard-based: streamlit, ...

# Imports

In [1]:
# magics:
# # autoreload imports. 
# # intent: if i change sth in import, i don't have to restart kernel. enable only for development.
%load_ext autoreload
%autoreload 2
# # choose matplotlib backend. backend 'notebook' allows interactive plots if your env allows it.
%matplotlib notebook


In [2]:
# python imports:
from collections import Counter
import time
#from pprint import pprint

In [3]:
#%pylab inline
#figuresize=(18, 4)
from collections import Counter
from math import pi
import pandas as pd
from bokeh.io import output_file,output_notebook, show
from bokeh.layouts import column
from bokeh.palettes import Category20c
from bokeh.plotting import figure
from bokeh.transform import cumsum
from bokeh.models import Legend,LegendItem

In [4]:
# aiida imports:
from aiida import load_profile
profile = load_profile('sisclab20')

# ggf add futher imports
from aiida.orm import QueryBuilder as QB
from aiida.orm import Node, User, CalcJobNode, Computer, Code
from aiida.plugins import DataFactory

from aiida.common.constants import elements as PeriodicTableElements

In [5]:
# project imports:
import helpers

# equivalent ('.' is the sisc_lab directory):
# from . import helpers
# alternative:
# from .helpers import print_bold
# from .helpers import * ('*' import everything; use of '*' is considered bad style)

ModuleNotFoundError: No module named 'helpers'

In [6]:
import sys
from pathlib import Path

## add path to system path
def add_to_sys_path(path:Path):
    if str(path) not in sys.path:
        sys.path.append(str(path))
        
path_johannes_comments = Path.cwd()
add_to_sys_path(path_johannes_comments.parent.parent)
import helpers

In [7]:
# (example:)
helpers.print_bold(f"This notebook/dashboard will visualize the contents from the database of profile {profile.name}")

This notebook/dashboard will visualize the contents from the database of profile sisclab20


# SubtaskD1.a: Node information

Task:

In [8]:
# query for all nodes
print('Information on nodes in the DB: \n')
now = time.strftime("%c")
print('last executed on {}'.format(now))
q = QB()
q.append(Node, project=['id', 'ctime', 'mtime', 'node_type'], tag='node')
q.append(User, with_node='node', project='email')
# TODO: execute query here
t = time.time()
elapsed = time.time() - t
res = q.all()
totalnodes = len(res)
print("Total number of nodes in the database: {} (retrieved in {} s.)".format(totalnodes, elapsed))

Information on nodes in the DB: 

last executed on Thu Jan  7 22:32:20 2021
Total number of nodes in the database: 5629 (retrieved in 3.147125244140625e-05 s.)


## Remarks Johannes

- Question: Who did `D1.a`?
- Time measurement measures nothing.

In [9]:
# just curious how result items look like
res[:2]

[[1,
  datetime.datetime(2017, 7, 4, 18, 46, 12, 127401, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=120, name=None)),
  datetime.datetime(2020, 10, 28, 13, 40, 52, 113135, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=60, name=None)),
  'data.code.Code.',
  'j.broeder@fz-juelich.de'],
 [2,
  datetime.datetime(2017, 8, 10, 11, 39, 53, 156133, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=120, name=None)),
  datetime.datetime(2020, 10, 28, 13, 42, 1, 387878, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=60, name=None)),
  'data.code.Code.',
  'j.broeder@fz-juelich.de']]

# SubtaskD1.b: Users

Task: print out a list of Users and how many nodes belong to them

for example

```
Users:
- j.broeder@fz-juelich.de created 182 nodes
- tests@aiida.mail created 104 nodes
```

In [10]:
users = Counter([r[4] for r in res])
print("Users:")
for count, email in sorted((v, k) for k, v in users.items())[::-1]:
    print("* {} created {} nodes".format(email, count))

Users:
* j.broeder@fz-juelich.de created 5629 nodes


## Remarks Johannes

- Question: Who did `D1.b`?

# SubtaskD1.c: Node types

Task: plot node information in two pie chart plots

One showing what data nodes there (with their lowest class names(node_type)) I.e Dict, K-pointsData, CifData, FleurinpData...

And one chart showning the process nodes, (with their lowest class names(process_type) i.e CalcjobNodes: FleurCalcjob, FleurinputgenCalcjob, ...

WorkChain nodes: FleurSCFWorkchain, FleurBandDosWorkchain, ..., calcfunctions, and workfunction nodes are fine to not show the lowest class names


In [11]:
#node types
types = Counter([r[3] for r in res])
print("Node types:")

for count, typestring in sorted((v, k) for k, v in types.items())[::-1]:
    print("* {}: {} nodes".format(typestring, count))

Node types:
* data.structure.StructureData.: 5169 nodes
* data.dict.Dict.: 194 nodes
* process.workflow.workfunction.WorkFunctionNode.: 49 nodes
* process.calculation.calcjob.CalcJobNode.: 43 nodes
* data.remote.RemoteData.: 43 nodes
* data.folder.FolderData.: 43 nodes
* data.fleur.fleurinp.FleurinpData.: 42 nodes
* process.workflow.workchain.WorkChainNode.: 28 nodes
* data.code.Code.: 18 nodes


In [12]:
#split data nodes and process nodes
labelst_1,labelst_2=[],[]
sizest_1,sizest_2=[],[]
#labelst = [label.split('.')[0]=='data' for label in types.keys()]
#sizest = [nnodes for nnodes in types.values()]
for k,v in types.items():
    if k.split('.')[0]=='data':
        labelst_1.append(k.split('.')[-2])
        sizest_1.append(v)
    elif k.split('.')[0]=='process':
        labelst_2.append(k.split('.')[-2])
        sizest_2.append(v)
        
#plot data nodes
#output_file("pie.html")
output_notebook()
x = dict(zip(labelst_1,sizest_1)) 
data=pd.DataFrame.from_dict(dict(x),orient='index').reset_index().rename(index=str,columns={0:'value','index':'data_nodes'})
data['angle'] = data['value']/sum(x.values()) * 2*pi
data['color'] = Category20c[len(x)]
p = figure(plot_height=350, title="Data Nodes", toolbar_location=None,
           tools="hover", tooltips="@data_nodes: @value")
p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='data_nodes', source=data)
p.axis.axis_label=None
p.axis.visible=False
p.grid.grid_line_color = None 
#show(p)                        

#plot process node

x1 = dict(zip(labelst_2,sizest_2)) 
data=pd.DataFrame.from_dict(dict(x1),orient='index').reset_index().rename(index=str,columns={0:'value','index':'process_nodes'})
data['angle'] = data['value']/sum(list(x1.values())) * 2*pi
data['color'] = Category20c[len(x1)]
p1 = figure(plot_height=350, title="Process Nodes", toolbar_location=None,
           tools="hover", tooltips="@process_nodes: @value")
p1.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='process_nodes', source=data)
p1.axis.axis_label=None
p1.axis.visible=False
p1.grid.grid_line_color = None 
show(column(p,p1))


Loading BokehJS ...

## Remarks Johannes

- First code cell:
  - Request: Remove full type path:
    - `data.structure.StructureData.: 5169 nodes` becomes `StructureData.: 5169 nodes`.
  - Hint: I personally find f-strings more readable: 
    - `f"There are {elephants} elephants."` is the same as `"There are {} elephants.".format(elephants)`.
- Second code cell:
  - Nice!
  - Suggestion: show percentages
  - Suggestion: Dashboarding:
    - checkbox combo list/...: select node types plotted

# SubtaskD1.d: Histogram

Task: Cumulative Histogram/ or line plot by ctime & mtime of all nodes over time

In [13]:
# line plot by ctime & mtime

ctimes = sorted(r[1] for r in res)
mtimes = sorted(r[2] for r in res)
num_nodes_integrated = range(len(ctimes))
df = pd.DataFrame({'A':ctimes,"B":mtimes})

#print(df.head())
#df = pd.DataFrame({'A':np.random.randn(100).cumsum(),"B":np.random.randn(100).cumsum()})

#plot multiline
p = figure(plot_width=900, plot_height=300, x_axis_type='datetime')
r=p.multi_line([df['A'], df['B']],  
               [df.index, df.index],   
               color=["firebrick", "navy"],   
               alpha=[0.8, 0.6],     
               line_width=[2,1],     
              )

legend=Legend(items=[
    LegendItem(label="ctime",renderers=[r],index=0),
    LegendItem(label="mtime",renderers=[r],index=1),
])
p.add_layout(legend)
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Number of nodes'

show(p)

## Remarks Johannes

- Suggestion: Dashboarding:
  - buttons/...: de/select mtime, ctime
  - slider/...: select daterange plotted

# SubtaskD1.e: Codes

Task: List Code names, sorted by by how many calcjobs where run with each

In [65]:
codes = Code.objects.all()
result = {code.label : len(code.get_outgoing(node_class=CalcJobNode).all_nodes()) for code in codes}
result_df=pd.Series(result).sort_values(ascending=False)
result_df

fleur_mpi_v0.28          22
inpgen                   21
fleur_mpi_max_1.3_dev     0
fleur_max_1.3_dev         0
fleur_mac_3_mpi           0
fleur_dev_mpi_booster     0
inpgen_max_2              0
fleur_max_2               0
fleur_dev_mpi             0
fleur_mpi_max_2           0
fleur_iff_0.28            0
inpgen_iff_0.28           0
inpgen_v0.28              0
fleur_v0.28               0
dtype: int64

## Remarks Johannes

- Request: as already discussed with Miao, the code names should contain the computer label as well, otherwise they are not unique. Look at how aiida prints a code node: 'codelabel on computerlabel':

In [79]:
codes[9]

<Code: Remote code 'inpgen_max_2' on local_mac, pk: 1713, uuid: 4c04ee95-f855-4654-98d5-3aab022335cc>

- Request: as already discussed with Miao, use a pandas *Dataframe* not a Series to get a nicer HTML rendering of the list. Like below. But if so, get rid of the row index.

In [83]:
result_df2 = pd.DataFrame({'code@computer':result.keys(), 'CalcJob count':result.values()})\
             .sort_values(by='CalcJob count', ascending=False)
result_df2

,code@computer,CalcJob count
6,fleur_mpi_v0.28,22
13,inpgen,21
0,fleur_v0.28,0
1,inpgen_v0.28,0
2,inpgen_iff_0.28,0
3,fleur_iff_0.28,0
4,fleur_mpi_max_2,0
5,fleur_dev_mpi,0
7,fleur_max_2,0
8,inpgen_max_2,0


# SubtaskD1.f: Groups

Task: List all group names with how many nodes they contain (verdi group list -C) (exclude import and export groups)

In [89]:
def ListGroup(Group : list, exclude : list):
    """ return the group names and nodes they contain
    
    :the list of all groups
    :the list of all excluded groups name
    """
    print('{:<52}{:6}'.format('Group names:','sizes:'))
    for a in Group:
        flag=0
        type = a[0].type_string
        for ex in exclude:
            if ex in type:
                flag=1
        if(flag):
            continue     
        else:
            ## the line below contains all the properties
            ##print(a[0].label,' ',a[0].user,' ',a[0].type_string,' ',a[0].description)

            print('{:<50}|{:5}'.format(a[0].label,len(a[0].nodes)))

In [90]:
load_profile()
#!verdi group list --all
from aiida.orm import QueryBuilder
from aiida.orm import load_node, Node, Group, Computer, User, CalcJobNode, Code, StructureData

qb = QueryBuilder()
qb.append(Group)
#print(qb.all())

group = qb.all()
ListGroup(group,['export','import'])

Group names:                                        sizes:
sisclab2020_small_database_201214                 | 5629
codes                                             |   18
fusion_materials                                  |   79
stable_binary_metals_from_materialsproject        | 5058
fleur_initial_cls_finished                        |    7
delta_parameters_gutstav_soc                      |   71
fusion_strucs_refinded                            |   23


## Remarks Johannes

- Suggestion: nice that you wrote a function for this. but in my view a HTML list via Dataframe would look nicer and is actually easier (see Remarks Johannes `D1.e` above). Do ASCII or HTML, but then be consistent throughout the notebook with it. But I would recommend HTML.
- Request: sorted list.
- Code style: 
  - [function naming style](https://google.github.io/styleguide/pyguide.html#s3.16-naming) here would be `list_groups(groups: list)`, not `ListGroup(Group : list)`. Also adjust naming in funcion body. The code is really hard to understand when it should be easy.
  - I don't get the reasoning behind `['export', 'import']` as a function agument. Why do you exclude these group types?

# SubtaskD1.g: Structures

Task: Further analyze what structures are in the DB

Number of structureData node versus how many atoms they contain. 

here interactive with bokeh hover tool showing the structure formula and uuid

Number of StructureData nodes versus elements bokeh bar chart, since there are over 
100 elements in the periodic table you can split it over several plots, or just use the charge number as in 
'example/element_content.png' but then make it interactive that once one hovers 
with the mouse over a bar it tells you what element it is and how many structures there are containing this element-

In [91]:
from aiida.orm import QueryBuilder
from aiida.orm import load_node, Node, Group, Computer, User, CalcJobNode, Code, StructureData
from HelperPackage import DataProcessing
from DataProcessing.DataVisu import StrucDataForm

def NumStructureNode():
    #### return the pd.DataFrame including elements and number of each element
    import pandas as pd
    StrucList = []
    qb = QueryBuilder()
    qb.append(StructureData)
    print('number of StructureData Nodes:',qb.count())
    qb.count()

    for struc, in qb.all()[:]:
        form = struc.get_formula()
        struct = StrucDataForm(form)
        StrucList = StrucList+ [struct.FormAnalyse()]
    return pd.DataFrame(StrucList).fillna(0)

def ShowingElements(Data):
    from bokeh.io import output_file, show
    from bokeh.models import ColumnDataSource
    from bokeh.plotting import figure
    from bokeh.io import output_notebook
    from bokeh.palettes import inferno

    output_file("ShowingElements.html")
    #data = NumStructureNode()
    data = Data
    elements = list(data.columns)
    counts = list(data.astype(bool).sum(axis=0))
    #print(counts)
    #print(elements)
    
    source = ColumnDataSource(data=dict(elements=elements, counts=counts,color=inferno(len(elements))))
    
    TOOLTIPS = [
    ("element", "@elements"),
    ("(x,y)", "($x, $y)"),
    ("Structures containing this element", "@counts"),
    ]

    p = figure( y_range=elements,x_range=(0,500), plot_width=800, plot_height=800, title="Elements Counts", tooltips=TOOLTIPS)
    #print('step figure done')
    p.hbar(y="elements", right="counts", height=0.5, left=0, color='color',  source=source)
    #print('step hbar done')
    
    output_notebook()
    p.xgrid.grid_line_color = None
    #p.legend = False
    show(p)
    
    
def ShowFormula(Data):
    from bokeh.io import output_file, show
    from bokeh.models import ColumnDataSource
    from bokeh.plotting import figure
    from bokeh.io import output_notebook
    from bokeh.palettes import inferno

    data = Data
    elements = list(data.keys())
    counts = list(len(data[key])/2 for key in data.keys())
    formulas = list(data[key] for key in data.keys())

    length = len(elements)
    source = ColumnDataSource(data=dict(elements=elements, counts=counts,color=inferno(length)))
    
    TOOLTIPS = [
    ("Number of Atoms", "@elements"),
    ("(x,y)", "($x, $y)"),
    ("Number of Nodes", "@counts"),
    ("id and formula", "@formulas"),
    ]

    p = figure( y_range=(0,160),x_range=(0,1000), plot_width=800, plot_height=800, title="Atoms Count", tooltips=TOOLTIPS)
    #print('step figure done')
    p.hbar(y="elements", right="counts", height=0.5, left=0, color='color',  source=source)
    #print('step hbar done')
    
    output_notebook()
    p.xgrid.grid_line_color = None
    #p.legend = False
    show(p)
    
def GetFormulaDict():
    
    ## may be not necessary, return the formulas and nodes containing this formula
    import numpy as np
    from aiida.orm import QueryBuilder,StructureData
    qb = QueryBuilder()
    qb.append(StructureData)
    Mydict = qb.all()
    Mydict = np.ravel(Mydict)

    Newdict = {}
    for dict in Mydict:
        if dict.get_formula in Newdict.keys():
            Newdict[dict.get_formula()].append(dict.uuid)
        else:
            Newdict[dict.get_formula()] = [dict.uuid]
    return Newdict

def AtomsNumNodes():
    from aiida.orm import QueryBuilder,StructureData
    import numpy as np
    qb = QueryBuilder()
    qb.append(StructureData)
    StructDatas = qb.all()
    Newdict = {}
    
    for data, in StructDatas:
        CompositionDict = data.get_composition()
        NumAtom = np.sum(list(CompositionDict.values()))
        if NumAtom in Newdict.keys():
            Newdict[NumAtom].append(data.uuid)
            Newdict[NumAtom].append(data.get_formula())
        else:
            Newdict[NumAtom] = [data.uuid,data.get_formula()]
    return Newdict

/Users/wasmer/src/aiida-jutools/aiida_jutools/sisc_lab/HelperPackage
__init__.py Initialization successful
/Users/wasmer/src/aiida-jutools/aiida_jutools/sisc_lab/HelperPackage
__init__.py Initialization successful


In [92]:
dic = AtomsNumNodes()
ShowFormula(dic)

Loading BokehJS ...

In [93]:
data = NumStructureNode()
ShowingElements(data)

number of StructureData Nodes: 5169


Loading BokehJS ...

In [97]:
# initialize the Structure data
from aiida.orm import QueryBuilder
from aiida.orm import load_node, Node, Group, Computer, User, CalcJobNode, Code, StructureData
from HelperPackage import DataProcessing
from DataProcessing.DataVisu import StrucDataForm

qb = QueryBuilder()
qb.append(StructureData)

structures = qb.all()

for structure in structures[::100]:
    formula = structure[0].get_formula()
    struct = StrucDataForm(formula)
    structure[0].get_composition()
    print(struct.FormAnalyse())

{'Cd': '3', 'Zr': '1'}
{'Si': '11', 'Yb': '8'}
{'C': '2', 'Ni': '6'}
{'B': '20', 'Na': '3'}
{'Dy': '4', 'Zn': '12'}
{'Hg': '2', 'Zr': '6'}
{'Bi': '6', 'Sm': '2'}
{'B': '4', 'Mo': '4'}
{'Ce': '2', 'Ga': '2'}
{'O': '58', 'W': '20'}
{'Ge': '6', 'Ta': '10'}
{'Mg': '2', 'Pb': '1'}
{'Ag': '2', 'Li': '6'}
{'O': '8', 'Ti': '7'}
{'Ga': '3', 'Pt': '5'}
{'Ni': '4', 'P': '12'}
{'Al': '4', 'Ce': '1'}
{'Ag': '1', 'Hf': '2'}
{'O': '1', 'Ti': '1'}
{'As': '1', 'Pa': '1'}
{'Ce': '2', 'Ga': '12'}
{'Er': '1', 'Sb': '1'}
{'Bi': '28', 'Rh': '6'}
{'La': '39', 'Se': '56'}
{'Fe': '1', 'V': '1'}
{'Ho': '8', 'Sb': '6'}
{'B': '16', 'U': '4'}
{'Ho': '3', 'Sm': '1'}
{'Be': '44', 'Re': '2'}
{'Rh': '2', 'Sn': '4'}
{'Sm': '1', 'Tl': '3'}
{'Np': '2', 'Si': '4'}
{'Hf': '1', 'Te': '2'}
{'Ni': '12', 'Ti': '4'}
{'Ag': '6', 'Ce': '2'}
{'In': '3', 'Ni': '2'}
{'B': '16', 'Ho': '4'}
{'B': '2', 'Ho': '1'}
{'Si': '3', 'U': '1'}
{'Au': '1', 'Tm': '1'}
{'Mn': '2', 'S': '4'}
{'Co': '4', 'Nb': '2'}
{'Cu': '13', 'I': '15'}
{'Cu': '6'

## Remarks Johannes

# SubtaskD1.h: Calculations

Task: more detail analysis of Calculations

`print('\n\nMore detailed analysis of Calculations \n')`

List, stacked Histogram of Calculations types and the state it ended up finished, failed, exit codes, exit messages

more detail analysis of WorkChains

`print('\n\nMore detailed analysis of WorkChains \n')`

List,  stacked Histogram for each Workchain type and the state it ended up in finished, failed, exit codes, exit messages

In [98]:
def GetCalNode():
    exit_state = []
    exit_message = []
    index = []
    exit_state_digit = []
    for node, in CalcNode:
        print(str(node.process_state))
        print(node.pk)
        print(str(node.exit_message))
        exit_state = exit_state + [str(node.process_state)]
        exit_message = exit_message + [str(node.exit_message)]
        index = index + [node.pk]
        if node.is_finished_ok:
            exit_state_digit = exit_state_digit + [1]
        else:
            exit_state_digit = exit_state_digit + [0]
    return exit_state,exit_message,index,exit_state_digit

def ShowCalNode(exit_state,exit_message,index,exit_state_digit):
    from bokeh.io import output_file, show
    from bokeh.models import ColumnDataSource
    from bokeh.plotting import figure
    from bokeh.io import output_notebook
    from bokeh.palettes import inferno

    output_file("ShowingCal.html")

    index = index
    exit_message = exit_message
    exit_state_string = exit_state
    exit_state_digit = exit_state_digit
      
    source = ColumnDataSource(data=dict(index=index, exit_state_digit=exit_state_digit,exit_message = exit_message,exit_state_string = exit_state_string, color=inferno(len(index))))
    
    TOOLTIPS = [
    ("Exit information", "@exit_message"),
    ("(x,y)", "($x, $y)"),
    ("Node index", "@index"),
    ("Node status", "@exit_state_string"),   
    ]

    p = figure( y_range=(0,1), x_range=(0,3000), plot_width=800, plot_height=800, title="CalcNode Information", tooltips=TOOLTIPS)
    #print('step figure done')
    p.vbar(x="index", top="exit_state_digit", bottom=0, width=1, color='color',  source=source)
    #print('step hbar done')
    
    output_notebook()
    p.xgrid.grid_line_color = None
    #p.legend = False
    show(p)
    
def GetWorkflowNode():
    return 0
    

In [99]:
from aiida.orm import CalcJobNode
from aiida.orm import QueryBuilder
qb = QueryBuilder()
qb.append(CalcJobNode)
CalcNode = qb.all()

exit_state,exit_message,index,exit_state_digit = GetCalNode()


ProcessState.FINISHED
4794
None
ProcessState.FINISHED
4781
None
ProcessState.FINISHED
4385
None
ProcessState.FINISHED
3925
None
ProcessState.FINISHED
3884
None
ProcessState.FINISHED
3239
None
ProcessState.FINISHED
3160
None
ProcessState.FINISHED
3094
None
ProcessState.FINISHED
2495
None
ProcessState.FINISHED
2436
None
ProcessState.FINISHED
2367
None
ProcessState.FINISHED
2198
None
ProcessState.FINISHED
2190
None
ProcessState.FINISHED
1788
None
ProcessState.FINISHED
1362
None
ProcessState.FINISHED
1304
None
ProcessState.FINISHED
1176
None
ProcessState.FINISHED
660
None
ProcessState.FINISHED
629
None
ProcessState.FINISHED
597
None
ProcessState.FINISHED
117
None
ProcessState.FINISHED
56
None
ProcessState.FINISHED
55
None
ProcessState.FINISHED
54
None
ProcessState.FINISHED
53
None
ProcessState.FINISHED
52
None
ProcessState.FINISHED
51
None
ProcessState.FINISHED
50
None
ProcessState.FINISHED
49
None
ProcessState.FINISHED
48
None
ProcessState.FINISHED
47
None
ProcessState.FINISHED
46
None
Pr

In [100]:
ShowCalNode(exit_state,exit_message,index,exit_state_digit)

Loading BokehJS ...

In [101]:
from aiida.orm import WorkflowNode
from aiida.orm import QueryBuilder
qb = QueryBuilder()
qb.append(WorkflowNode)
WNode = qb.all()

Newdict = {}
for node, in WNode:
    if node.is_finished_ok:
        Newdict[node.node_type+'_succeed'] = Newdict.get(node.node_type+'_succeed',0) + 1
        Newdict[node.node_type+'_not_succeed'] = Newdict.get(node.node_type+'_succeed',0) + 0
    else:
        Newdict[node.node_type+'_not_succeed'] = Newdict.get(node.node_type+'_not_succeed',0) + 1
        Newdict[node.node_type+'_succeed'] = Newdict.get(node.node_type+'_succeed',0) + 0
    #print(node.exit_message)
    #print(node.exit_status)
    #print(node.)
print(Newdict)

{'process.workflow.workchain.WorkChainNode._not_succeed': 28, 'process.workflow.workchain.WorkChainNode._succeed': 0, 'process.workflow.workfunction.WorkFunctionNode._not_succeed': 49, 'process.workflow.workfunction.WorkFunctionNode._succeed': 0}


In [102]:
def ShowWorkflow(WorkflowDict):
    from bokeh.io import output_file, show
    from bokeh.models import ColumnDataSource,HoverTool
    from bokeh.plotting import figure
    from bokeh.io import output_notebook
    from bokeh.palettes import inferno

    output_file("ShowingCal.html")

    index = list(WorkflowDict.keys())
    counts = list(WorkflowDict.values())
    #exit_message = exit_message
    #exit_state_string = exit_state
    #exit_state_digit = exit_state_digit
      
    source = ColumnDataSource(data=dict(index=index, counts=counts, color=inferno(len(index))))
    
    TOOLTIPS = [
    ("Node number", "@counts"),
    ("(x,y)", "($x, $y)"),
    ("Node status", "@index"),   
    ]
   
    HT = HoverTool(
    tooltips=TOOLTIPS,

    mode='vline'
    )
    
    p = figure( y_range=(0,50), x_range=index, plot_width=800, plot_height=800, title="CalcNode Information",tools = [HoverTool(mode='vline')],tooltips=TOOLTIPS)
    #print('step figure done')
    p.vbar(x="index", top="counts", bottom=0, width=1, color='color',  source=source)
    #print('step hbar done')
    
    output_notebook()
    p.xgrid.grid_line_color = None
    #p.legend = False
    show(p)

ShowWorkflow(Newdict)

Loading BokehJS ...

## Remarks Johannes

# SubtaskD1.i: Provenance

Task: Database and provenance health: display the number of nodes who have no incomming and outgoing links, no incomming links (any number outgoing), and no outgoing links (any number incomming)

## Remarks Johannes